## Scraping Historical S&P/TSX Composite Constituents Using Wikipedia
By Samuel Poon | www.elmrillinvestments.com | [GitHub](https://github.com/samuel-poon)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import dateutil.parser as dparser
import string
import pyprind
import datetime
import pandas_market_calendars as mcal

In [ ]:
edit_history = 'https://en.wikipedia.org/w/index.php?title=S%26P/TSX_Composite_Index&offset=&limit=500&action=history'
r = requests.get(edit_history)
soup = BeautifulSoup(r.content,'lxml')

historical_pages = soup.find_all('a', href=True)
historical_pages = list(filter(lambda a: 'title=S%26P/TSX_Composite_Index&oldid=' in a['href'],
                        historical_pages))
historical_pages = list(map(lambda a: 'https://en.wikipedia.org' + a['href'],
                            historical_pages))

historical_pages = historical_pages[:200]
historical_pages

We define two functions to pick up the constituents list and the day at which the index changes. *get_constituents* returns a DataFrame with the constituents while *get_as_of_date* returns the index change date in a datetime object.

In [ ]:
def get_constituents(url):
    try:
        # keep_default_na to False since National Bank's ticker is NA
        tables = pd.read_html(url, keep_default_na=False)

        # We only want to return the table with the constituents.
        for table in tables:
            if table.columns[0] == 'Symbol':
                
                # Some tables have four columns if they include the industry and some only include
                # the sector.
                if len(table.columns) == 4:
                    table.columns = ['Symbol','Company','Sector','Industry']
                elif len(table.columns) == 3:
                    table.columns = ['Symbol','Company','Sector']
                
                # On edits prior to April 12, 2012 edit, editors began adding 'TSX :' in the Symbol
                # column.
                table['Symbol'] = table['Symbol'].apply(lambda x: x.replace('TSX:\xa0', ''))
                
                return table
    except:
        return None

def get_as_of_date(url):
    try:
        wiki_r = requests.get(url)
        wiki_soup = BeautifulSoup(wiki_r.content, 'lxml')

        header = wiki_soup.find('span', {'class':'mw-headline'}, text=re.compile('^List of companies'))
        p = header.find_next('p').text
        
        # We want to remove all punctuation from the paragraph - sometimes this throws off
        # the parser.
        table = str.maketrans(dict.fromkeys(string.punctuation))
        p = p.translate(table)
        
        return dparser.parse(p, fuzzy=True)
    except:
        return None

tsx_history stores the constituents every time the index changes in a DataFrame. When there are multiple edits for the same day, **I prioritize the most recent Wikipedia edit**.

In [ ]:
last_date = get_as_of_date('https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index')
tsx_history = get_constituents('https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index')
tsx_history['Last Index Change'] = last_date

for url in pyprind.prog_bar(historical_pages):
    if not get_as_of_date(url) is None and get_as_of_date(url) != last_date:
        updated_constituents = get_constituents(url)
        updated_constituents['Last Index Change'] = get_as_of_date(url)

        tsx_history = tsx_history.append(updated_constituents,sort=False)
        last_date = get_as_of_date(url)

tsx_history.set_index('Last Index Change',inplace=True)
tsx_history

In [ ]:
index_change_dates = tsx_history.index.unique().sort_values(ascending=False).tz_localize(None)
index_change_dates

*get_historical_constituents* allows us to pick up the S&P/TSX Composite's constituents over a date range. Uses *tsx_history* to identify when the index changes. Returns output in a DataFrame. By default, *get_historical_constituents()* will return the constituent list from the latest trading day.

In [ ]:
def get_historical_constituents(start_date=datetime.date.today(), end_date=datetime.date.today()):

    date_range = mcal.get_calendar('TSX').valid_days(start_date, end_date).tz_localize(None)
    
    # If the market is not open today, then we want to pull the constituent list from the last trading day.
    # To find this, we look at valid_days over the last week and choose the last one. This is arbitrary
    # but should cover every case including weekends and holidays.
    if len(date_range) == 0:
        date_range = mcal.get_calendar('TSX').valid_days(datetime.date.today()-datetime.timedelta(days=7), datetime.date.today())
        date_range = date_range[len(date_range)-1:]
        date_range = date_range.tz_localize(None)

    output_df = pd.DataFrame()
    
    for trading_day in date_range:
        # Since index_change_dates is already sorted, we can just take the first value where
        # trading_day is equal to or less than the values in index_change_dates to find the 
        # most recent index change.
        last_index_change = list(filter(lambda x: x <= trading_day, index_change_dates))[0]
        
        constituents = tsx_history.loc[last_index_change].copy()
        constituents['Last Index Change'] = constituents.index
        constituents['Date'] = trading_day
        
        output_df = output_df.append(constituents)
    
    output_df.set_index('Date', inplace=True)
    
    return output_df

In [ ]:
df = get_historical_constituents(start_date='2020-01-01', end_date='2020-12-31')
df

We might want this data in a format where the rows are the dates and the columns are the symbols, with each value representing whether the stock was included in the index as of that date or not (useful for backtesting).

In [ ]:
df = df[['Symbol']]
df['Value'] = True
df.pivot(columns='Symbol', values='Value').fillna(False)

Saving the data in a csv for future use...

In [ ]:
get_historical_constituents().to_csv('GSPTSE_constituents.csv')
tsx_history.to_csv('GSPTSE_changes.csv')